# Solar Panels Customer Support Agent 

This notebook implements an enhanced example of customer support agent for solar panel products that utilizes these key features:

1. **Amazon Bedrock Knowledge Base**: Provides information about solar panel models, installation, and maintenance through Retrieval Augemented Generation
2. **Amazon Bedrock Guardrails**: Safeguards the Strands Agent and helps adhere to content policies
3. **Amazon DynamoDB Customer Profiles**: Stores and retrieves customer information from Amazon DynamoDB
4. **JIRA Integration**: Creates and manages support tickets in JIRA
5. **Mem0 Memory**: Provides long-term conversation memory for personalized interactions across sessions.
6. **Custom Support Tools**: Specialized tools for solar system performance analysis and warranty checks
7. **Observability with Langfuse**: Strands Agents built-in support for observability with LangFuse

The Strands Agent will identify the customer via their email address, retrieve their profile information from Amazon DynamoDB, create support tickets in JIRA when needed, and provide personalized support based on conversation history stored in Mem0.

## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Native tools used   |retrieve, mem0_memory                                         |
|Custom tools JIRA   |create_solar_support_ticket, get_customer_tickets |
|Custom tools Profile Management   |get_customer_profile, update_customer_profile|
|Custom tools Solar Panels   |analyze_solar_system_performance,check_warranty_status |
|Agent Structure     |Single agent architecture                          |
|AWS services used   |Amazon Bedrock Knowledge Base, Amazon DynamoDB, Amazon Bedrock Guardrails|
|Integrations        | Mem0, Langfuse, Atlassian JIRA                   |

</div>

## Architecture

<div style="text-align:left">
    <img src="images/architecture.png" width="75%" />
</div>



## Prerequisites

- Python 3.10+
- Anthropic Claude 3.7 enabled on Amazon Bedrock
- solar_panel_support_config.json populated with resources from 01_solar_panel_customer_support_setup.ipynb notebook.

| Component | Description |
|-----------|-------------|
| AWS Account | With access to Amazon Bedrock and Amazon DynamoDB |
| Mem0 API Key | For conversation memory storage (m0-*) |
| JIRA Account Details | For ticket creation and management  |
| Langfuse Account Keys| Public Key and Secret key for Observability |
| Python 3.10+ | With required packages in requirements.txt |

## 1. Install Required Dependencies

In [ ]:
# Install prerequisites

!pip install -r requirements.txt
#!pip install --upgrade strands-agents strands-agents-tools
#!pip install --upgrade -q boto3

## 2. Import Libraries and Setup

In [ ]:
import os
import time
import re
import json
import uuid
import boto3
import logging
import warnings
from pathlib import Path
import datetime
import base64
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import retrieve, mem0_memory
from langfuse import Langfuse
from dotenv import load_dotenv


# Import our custom tools for customer profile management with DynamoDB
from customer_profile_tools_dynamodb import get_customer_profile, update_customer_profile
from customer_profile_tools_dynamodb import analyze_solar_system_performance, check_warranty_status
# Import JIRA tools
from utils.jira_tools import create_solar_support_ticket, get_customer_tickets
from utils.customer_dynamodb import SolarCustomerDynamoDB

# Silence deprecation warnings
warnings.filterwarnings("ignore", message="output_format=.*")
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Set up logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')

## 3. Load Configuration

We'll load the Amazon Knowledge Base ID, Guardrail ID and Amazon DynamoDB ID from our configuration file that was created during setup notebook.

In [ ]:
# Load configuration from file
config_path = "solar_panel_support_config.json"

with open(config_path, 'r') as f:
    config = json.load(f)
    
print(f"Loaded configuration from {config_path}")

# Set up environment variables and configuration
GUARDRAIL_ID = config.get("guardrail", {}).get("id")
KB_ID = config.get("knowledge_base", {}).get("id")
REGION = config.get("region", "us-east-1")
CUSTOMER_TABLE = config.get("customer_table")

# Set environment variables for tools
os.environ["KNOWLEDGE_BASE_ID"] = KB_ID
os.environ["AWS_REGION"] = REGION

print(f"Guardrail ID: {GUARDRAIL_ID}")
print(f"Knowledge Base ID: {KB_ID}")
print(f"Region: {REGION}")
print(f"Customer Table: {CUSTOMER_TABLE}")

## 5. Display Available Test Customers

Let's list the available test customers from the Amazon DynamoDB table.

In [ ]:
# Query the table to get all customer profiles
dynamodb = boto3.resource('dynamodb', region_name=REGION)
table = dynamodb.Table(CUSTOMER_TABLE)
response = table.scan(Limit=10)  # Get up to 10 profiles

# Display available test customers
customer_list = response.get('Items', [])

print("Available test customers:")
for i, profile in enumerate(customer_list):
    print(f"- {profile.get('name')}: ID={profile.get('customer_id')}, Email={profile.get('email')}")
    
# Show details of first customer
if customer_list:
    sample = customer_list[0]
    print(f"\nSample profile details for {sample.get('name')}:")
    print(f"Customer ID: {sample.get('customer_id')}")
    print(f"Email: {sample.get('email')}")
    print(f"Country: {sample.get('country')}, State: {sample.get('state')}")
    print(f"Purchase history: {len(sample.get('purchase_history', []))} items")

# Initialize the DynamoDB client
db = SolarCustomerDynamoDB(region_name=REGION)

## 6. Define System Prompt

We'll create a comprehensive system prompt that guides the agent on how to interact with customers using all the integrated tools.

In [ ]:
# System prompt for the solar panel agent with memory, DynamoDB and JIRA integration
system_prompt = """
You are a helpful solar panel customer support assistant with access to customer profiles in DynamoDB, 
JIRA ticketing system, and memory of past conversations using mem0.

When interacting with customers:
1. First identify the customer using their email address (preferred) or customer ID
2. Access past conversation memory using mem0_memory to provide personalized continuity
3. Remember important details shared by the customer in previous conversations

Use the knowledge base to answer questions about solar panels, including installation, maintenance, pricing, 
efficiency, and technical specifications. When a customer asks about their specific solar panels or previous 
issues, refer to:
- Their purchase history (available in the customer profile under the 'purchase_history' field)
- Previous conversations stored in memory
- JIRA tickets if they exist

Always personalize your responses based on all available information. Use the customer's name, mention 
specific products they own, reference their past issues, and acknowledge previous conversations when appropriate.
Do not answer about other customers. Customer X cannot access customer Y data: respond with - This request is denied for privacy.

You have specialized tools available to help customers:

For Memory Management:
1. mem0_memory: To store and retrieve conversation history

For DynamoDB Access:
2. get_customer_profile: To retrieve customer details (including their purchase history) using email or ID
3. update_customer_profile: To update customer information

For Solar Analysis:
4. analyze_solar_system_performance: To analyze a customer's solar system performance
5. check_warranty_status: To check warranty status for customer products

For JIRA Ticketing:
6. create_solar_support_ticket: To create new support tickets in JIRA
   - When creating tickets, include the customer_email parameter when available
   - Always include the customer ID for proper tracking
7. get_customer_tickets: To retrieve customer tickets from JIRA


For Knowledge Base Access:
8. retrieve: To search the knowledge base for solar panel information

Create JIRA tickets when customers report issues that cannot be immediately resolved. Tickets should include:
- Descriptive title
- Problem description
- Customer ID
- Relevant product information

Store important information from each conversation in mem0_memory so you can refer to it in future interactions.
Always retrieve relevant memories at the beginning of the conversation to provide continuity.

Always be polite, professional, helpful, and empathetic. Provide clear and concise information.
"""

## Create Bedrock Model with Guardrails

We'll create a Bedrock model that uses Claude 3.5 Sonnet with the specified guardrail.

In [ ]:
# Create a Bedrock model with guardrail configuration
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    guardrail_id=GUARDRAIL_ID,              # Bedrock guardrail ID
    guardrail_version="DRAFT",              # Guardrail version
    guardrail_trace="enabled",              # Enable trace info for debugging
    boto_session=boto3.Session(region_name=REGION),
    guardrail_redact_output=True,           # Redact sensitive data in output
    guardrail_redact_input=True             # Redact sensitive data in input
)

## 7. Integration Setup

This section covers the setup for our three key integrations:
1. OpenTelemetry (OTEL) for observability with Langfuse
2. Mem0 for conversation memory
3. JIRA for ticket management

Each integration requires specific configuration and environment variables.

### 7.1 OpenTelemetry (OTEL) Setup

OpenTelemetry provides standardized observability for our agent, allowing us to track interactions, performance, and errors. We'll use Langfuse as our OpenTelemetry endpoint for visualization and analysis.

**Required Environment Variables:**
- `LANGFUSE_HOST`: The Langfuse API endpoint
- `OTEL_EXPORTER_OTLP_ENDPOINT`: Where to send OpenTelemetry data
- `OTEL_EXPORTER_OTLP_HEADERS`: Authentication for the endpoint

**How OTEL Works with Strands:**
1. Each agent interaction generates spans and traces
2. The trace data is sent to Langfuse via OTLP (OpenTelemetry Protocol)
3. This enables analysis of agent performance, latency, and behavior

**Langfuse:** [Langfuse](https://langfuse.com/docs/integrations/strands-agents) is an open-source LLM engineering platform. It provides robust tracing, debugging, evaluation, and monitoring capabilities for AI agents and LLM applications.

Please add your public key and secret key from API keys section in Langfuse settings in the below cell. 

In [ ]:
# Get keys for your project from the project settings page: https://cloud.langfuse.com

os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-your-api-key"
os.environ["LANGFUSE_SECRET_KEY"] = "sk-your-api-key" 
#os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region (default)
os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region
 
# Build Basic Auth header.
LANGFUSE_AUTH = base64.b64encode(
    f"{os.environ.get('LANGFUSE_PUBLIC_KEY')}:{os.environ.get('LANGFUSE_SECRET_KEY')}".encode()
).decode()
 
# Configure OpenTelemetry endpoint & headers
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = os.environ.get("LANGFUSE_HOST") + "/api/public/otel/v1/traces"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

### 7.2 Mem0 Memory Integration

[Mem0](https://mem0.ai/) provides long-term conversation memory for our agent, allowing it to recall previous interactions with users. This notebook uses [Strands-agents-tools](https://github.com/strands-agents/tools/tree/main/src/strands_tools) in-built [mem0-memory](https://github.com/strands-agents/tools/blob/main/src/strands_tools/mem0_memory.py) tool. 

**Required Environment Variables:**
- `MEM0_API_KEY`: Your Mem0 API key for authentication

**How Mem0 Works with Strands:**
1. Important conversation details are stored in Mem0 using the `mem0_memory` tool
2. Each memory is associated with a specific user_id ( email)
3. The agent can retrieve past conversation memories to provide continuity

In [ ]:
os.environ["MEM0_API_KEY"] = "your-api-key"

### 7.3 JIRA Integration Setup

JIRA integration allows our agent to create and manage support tickets directly. This provides a seamless way to track customer issues that require follow-up. Create a project and update the name in the .env file.

**Required Environment Variables (stored in .env file):**
- `JIRA_INSTANCE_URL`: Your JIRA instance URL (e.g., "https://your-domain.atlassian.net/")
- `JIRA_USERNAME`: Your JIRA account email
- `JIRA_API_TOKEN`: API token generated from your Atlassian account
- `JIRA_CLOUD`: "True" if using Jira Cloud (vs. Server)
- `PROJECT_NAME`: Name of your JIRA project for tickets (e.g., "SOLAR")

**How Environment Variables Work:**
1. These variables are stored in a `.env` file in the project root directory
2. The `.env` file is loaded using `python-dotenv` package
3. This keeps sensitive credentials out of code and notebooks
4. Variables can be accessed via `os.getenv("VARIABLE_NAME")`

**Example .env.example file structure:**
```
JIRA_INSTANCE_URL=https://your-domain.atlassian.net/
JIRA_USERNAME=your-email@example.com
JIRA_API_TOKEN=your-api-token
JIRA_CLOUD=True
PROJECT_NAME=SOLAR
```

**How JIRA Integration Works:**
1. The agent uses the `create_solar_support_ticket` tool to create tickets
2. The `get_customer_tickets` tool retrieves existing tickets for a customer
3. Each ticket includes customer details and issue description
4. The JIRA API handles the actual ticket creation and management

In [ ]:

# Load environment variables from .env file
load_dotenv()

# Print JIRA configuration (masking sensitive data)
print("\n===== JIRA Configuration =====\n")
print(f"JIRA URL: {os.getenv('JIRA_INSTANCE_URL')}")
print(f"JIRA USERNAME: {os.getenv('JIRA_USERNAME')}")
print(f"JIRA API TOKEN: {'*****' if os.getenv('JIRA_API_TOKEN') else 'Not set'}")
print(f"JIRA CLOUD: {os.getenv('JIRA_CLOUD')}")
print(f"PROJECT NAME: {os.getenv('PROJECT_NAME', 'SOLAR')}")

# Note: If you're seeing "None" for these values, make sure your .env file is:
# 1. Created in the project root directory
# 2. Contains the correct variable names and values

## 8. Create the Support Agent

Now we'll create the Strands Customer Support Agent with all the integrated tools.

In [ ]:
# Create the agent with all integrated tools
support_agent = Agent(
    system_prompt=system_prompt,
    model=bedrock_model,
    callback_handler=None,
    tools=[
        # Memory tools
        mem0_memory,
        # DynamoDB customer profile tools
        get_customer_profile,
        update_customer_profile,
        
        # Solar analysis tools
        analyze_solar_system_performance,
        check_warranty_status,
        
        # JIRA integration tools
        create_solar_support_ticket,
        get_customer_tickets,
        
        # Knowledge base and web search
        retrieve
        
    ],
    trace_attributes={
        "session.id": "abc-1234", # Example session ID
        "user.id": "user-email-example@domain.com", # Example user ID
        "langfuse.tags": [
            "dev",
            "Strands-Project-Demo",
            "Observability-Tutorial",
        ]
    }
    
)

In [ ]:
# Define a test customer email - use one from the list displayed earlier
test_customer_email = "customer1@example.com"

# Define a function to test the agent with a specific question
def test_agent_question(question):
    """Test the agent with a single question."""
    print(f"\n📝 QUESTION: {question}\n")
    
    # Build context with the customer email
    context = f"The customer's email is {test_customer_email}. "
    full_question = context + "\n\n" + question
    
    # Start timing for performance tracking
    start_time = time.time()
    
    try:
        # Get response from the agent
        response = support_agent(full_question)
        
        # Calculate response time
        response_time = time.time() - start_time
        print(f"\n🤖 RESPONSE:\n{response}")
        print(f"\n⏱️ Response generated in {response_time:.2f} seconds\n")
        print("=" * 80)
        #return response
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        print("The model may have encountered an issue processing your request.")
        return None

## Test the Agent tools without the memory capability 
### Test 1: Customer Profile Question
Test retrieving customer profile information

In [ ]:
#Test agent retrieving customer profile information
test_agent_question("What solar panels do I have installed? When did I purchase them?")

### Test 2: Warranty Status Check
Test checking warranty status for customer's solar panels

In [ ]:
# Test checking warranty status
test_agent_question("Is my solar panel still under warranty? How long do I have left on the warranty?")

### Test 3: System Performance Analysis

Test analyzing the solar system's performance

In [ ]:
# Test performance analysis
test_agent_question("How is my solar system performing? Is it producing the expected amount of energy?")

### Test 4: Knowledge Base Query
Test retrieving information from the solar panel knowledge base

In [ ]:
# Test knowledge base retrieval
test_agent_question("How do I clean my SunPower X solar panels properly? What cleaning products should I use?")

### Demonstrating how to store memory using the mem0_memory personalised response.

In [ ]:
# First, let's store a memory about the customer's preference
memory_content = "The customer mentioned they prefer to do maintenance on weekends and is interested in upgrading their inverter next year."

# Store the memory
try:
    result = support_agent.tool.mem0_memory(
        action="store", 
        content=memory_content, 
        user_id=test_customer_email,
        metadata={"timestamp": datetime.datetime.now().isoformat()}
    )
    print("✅ Memory stored successfully")
except Exception as e:
    print(f"❌ Error storing memory: {e}")
    
# Now let's test if the agent retrieves and uses this memory in its response
test_agent_question("When would be a good time to schedule my annual maintenance check?")

## 9. Helper Functions

These functions will help manage customer interactions and handle memory storage and retrieval.

In [ ]:
# Define common regex patterns once
EMAIL_REGEX = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

# Function to retrieve memories for a customer
def retrieve_customer_memories(customer_email):
    """Retrieve conversation memories for a customer using the mem0_memory tool."""
    try:
        # Use the agent to call the mem0_memory tool
        results = support_agent.tool.mem0_memory(action="list", user_id=customer_email)
        
        # Check if we have any memories
        if isinstance(results, dict) and results.get('status') == 'error':
            print("No previous memories found. Starting fresh conversation.")
            return []
            
        # Format the results
        if isinstance(results, list):
            return results
        elif isinstance(results, dict) and 'results' in results:
            return results['results']
        else:
            return []
    except Exception as e:
        print(f"Error retrieving memories: {str(e)}")
        return []

# Function to store a conversation in memory
def store_conversation_memory(customer_email, query, response):
    """Store a conversation interaction in mem0."""
    try:
        # Format the conversation entry
        memory_entry = f"User: {query}\nAssistant: {response}"
        
        # Add to mem0 using the agent's tool
        result = support_agent.tool.mem0_memory(
            action="store", 
            content=memory_entry, 
            user_id=customer_email,
            metadata={"timestamp": datetime.datetime.now().isoformat()}
        )
        print("✓ Conversation stored in memory")
        return True
    except Exception as e:
        print(f"Error storing memory: {str(e)}")
        return False

# Function to ask a question to the support agent
def ask_support_question(question, customer_email):
    """Ask the support agent a question with customer context detection.

    Args:
        question (str): The customer support question
        customer_email (str): The customer's email address

    Returns:
        str: Response from the agent
    """
    # Build context message to include with the question
    context = f"The customer's email is {customer_email}. "
    
    # Get customer memories and add to context
    memories = retrieve_customer_memories(customer_email)
    if memories:
        memory_context = "\n\nHere are relevant past conversations with this customer:\n"
        
        # Include only the most recent 3 memories for context
        for i, memory in enumerate(memories[:3]):
            memory_content = memory.get('memory', '')
            memory_context += f"Memory {i+1}: {memory_content}\n\n"
            
        context += memory_context
    
    # Combine context and question
    full_question = context + "\n\n" + question
    
    # Display processing message
    print("\nProcessing your question...\n")
    
    # Start timing for performance tracking
    start_time = time.time()
    
    # Get response from the agent
    try:
        response = support_agent(full_question)
        
        # Store this conversation in memory
        store_conversation_memory(customer_email, question, response)
        
        # Calculate response time
        response_time = time.time() - start_time
        print(f"\n(Response generated in {response_time:.2f} seconds)\n")
        
        return response
    except Exception as e:
        print(f"\nError: {str(e)}")
        print("The model may have encountered an issue processing your request.")
        return None

### 10. Solar panels customer support with memory and ticket creation to answer questions based on contextual memory using mem0 storage and retrieval


Now let's demonstrate the support agent session for a user.

In [ ]:
print("🌐 Solar Panel Customer Support Agent 🌐\n")
print("This agent helps with solar panel customer support questions.")
print("It will identify you by your email, access your profile from DynamoDB,")
print("remember past conversations, and create JIRA tickets when needed.\n")


customer_email= "customer5@example.com"


query = "My SunPower X panel is showing blinking red light and isn't producing any power. Create a support ticket for it."

response = ask_support_question(query, customer_email)
if response:
    print(f"\n{response}")

In [ ]:
query = "What's the status of my support tickets? Do I have any open tickets?"

response = ask_support_question(query, customer_email)
if response:
    print(f"\n{response}")


### Amazon Bedrock Guardrail Test

In [ ]:
query = "Should I spend to invest in the stock market for better returns? What stocks would you recommend for green energy investments?"

response = ask_support_question(query, customer_email)
if response:
    print(f"\n{response}")


# Congratulations! 
You created a Customer support bot with various features using Strands Agents SDK. For the Agent builder to monitor the interactions from the Customer support bot for observability, the traces are logged in Langfuse using the environment variables you have defined. When you open your Langfuse dashboard, you can see the details on the input questions, tool use and the output in the traces, token usage, latency etc. 



# JIRA board 

In your domain in Atlassian, under your project you will be able to see the tickets created and you can update, assign them directly in JIRA project that you created. 

![JIRA Tickets](images/JIRA.png)

# Langfuse Dashboard

In the Langfuse Dashboard you will be able to view the Traces in detail. The trace attributes, such as session.id, user.id, and langfuse.tags, are sent to Langfuse with the traces and help organize, filter, and analyze traces in the Langfuse UI.

![Langfuse Traces](images/langfuse_traces.png)

# Mem0 Dashboard

In the mem0 Dashboard you will be able to view the conversations stored as shown in the image below.

![Mem0 dashboard](images/mem0_memories.png)

## 11. Example Questions for Each Integrated Component

Below are example questions that demonstrate each of our integrated tools. You can try these or similar questions in the interactive session above.

### Profile Questions

- "Can you tell me what solar panels I have?"
- "What's my purchase history?"
- "When did I buy my solar system?"
- "What are my contact preferences?"

### Solar System Performance Questions

- "How is my solar system performing?"
- "Can you analyze my system performance for the last month?"
- "What's the efficiency of my solar panels?"

### Warranty and Maintenance Questions

- "What's the warranty status on my SunPower X panels?"
- "How do I submit a warranty claim?"
- "When does my panel warranty expire?"
- "What's covered under my solar panel warranty?"

### JIRA Ticket Questions

- "I'm having issues with my SunPower X panels with battery drainage, can you create a ticket?"
- "What's the status of my support tickets?"
- "I need someone to come fix my solar panel, it's showing error code E5."
- "Can you look up my previous support tickets?"

### Product Manual Questions

- "How do I clean my SunPower X panels?"
- "What's the installation process for SunPower Y panels?"
- "What maintenance do solar panels need?"
- "What are the features of the SunPower Double-X model?"

### Memory-Based Personalization Questions

- "What did we discuss last time?"
- "You mentioned something about panel cleaning before, what was that?"
- "What recommendation did you give me for improving efficiency?"


## 12. Clean up

When you're done with this notebook, you can clean up the resources.

In [ ]:
# Clean up resources when you're done with experimenting this example


# Import needed utilities
import boto3
from utils.customer_dynamodb import SolarCustomerDynamoDB
from utils.knowledge_base import BedrockKnowledgeBase
import sys
sys.path.append('utils')

print("📋 Starting cleanup of resources...")

# Initialize resources
db = SolarCustomerDynamoDB(region_name=REGION)
bedrock_client = boto3.client('bedrock', region_name=REGION)
s3_client = boto3.client('s3', region_name=REGION)

# 1. Delete DynamoDB table
if CUSTOMER_TABLE:
    print(f"🗃️ Cleaning up DynamoDB table {CUSTOMER_TABLE}...")
    
    # First verify table actually exists
    if db.table_exists(CUSTOMER_TABLE):
        result = db.delete_table(CUSTOMER_TABLE)
        if result:
            print(f"✅ DynamoDB table and SSM parameter cleaned up successfully")
        else:
            print(f"❌ Failed to clean up DynamoDB table")
    else:
        print(f"ℹ️ Table {CUSTOMER_TABLE} doesn't exist, cleaning up SSM parameter only")
        
        # Clean up SSM parameter even if table doesn't exist
        try:
            ssm_client = boto3.client('ssm', region_name=REGION)
            ssm_client.delete_parameter(Name='solar-customer-table-name')
            print("✅ Cleaned up SSM parameter")
        except Exception as e:
            if 'ParameterNotFound' in str(e):
                print("ℹ️ SSM parameter doesn't exist")
            else:
                print(f"❌ Error cleaning up SSM parameter: {e}")

# 2. Delete Knowledge Base and related resources
if KB_ID:
    print(f"📚 Cleaning up Knowledge Base {KB_ID}...")
    # Create boto3 client to delete KB directly
    bedrock_agent_client = boto3.client('bedrock-agent', region_name=REGION)

    try:
        # List and delete all data sources first
        data_sources =bedrock_agent_client.list_data_sources(knowledgeBaseId=KB_ID)["dataSourceSummaries"]
        print(f"Found {len(data_sources)} data sources to delete")

        for ds in data_sources:
            print(f"Deleting data source: {ds['dataSourceId']}")
            bedrock_agent_client.delete_data_source(
                dataSourceId=ds["dataSourceId"],
                knowledgeBaseId=KB_ID
            )

        # Then delete the knowledge base
        print(f"Deleting knowledge base with ID: {KB_ID}")
        bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=KB_ID)
        print(f"✅ Successfully deleted knowledge base")
    except Exception as e:
        print(f"Error during deletion: {str(e)}")

# 3. Delete Guardrail
if GUARDRAIL_ID:
    print(f"🛡️ Cleaning up Guardrail {GUARDRAIL_ID}...")
    try:
        bedrock_client.delete_guardrail(guardrailIdentifier=GUARDRAIL_ID)
        print(f"✅ Guardrail deletion initiated. This may take several minutes to complete.")
    except Exception as e:
        print(f"❌ Error cleaning up Guardrail: {e}")

print("🧹 Cleanup process completed")


## Summary and Conclusion

This notebook demonstrated how to build an example of a comprehensive solar panel customer support agent that integrates multiple platforms and services:

### Key Components Implemented

1. **Amazon Bedrock Knowledge Base Integration**
   - Provides solar panel product information, installation guides, and maintenance procedures
   - Enables context-aware responses using domain-specific knowledge

2. **Amazon Bedrock Guardrails**
   - Safe and appropriate responses
   - Prevents prohibited content like investment advice
   - Maintains professional customer interactions

3. **Amazon DynamoDB Customer Profiles**
   - Stores and retrieves customer information
   - Enables personalized support based on purchase history
   - Maintains customer preferences and details

4. **JIRA Integration**
   - Creates support tickets for issues requiring follow-up
   - Retrieves existing ticket information
   - Provides structured tracking of customer issues

5. **Mem0 Memory**
   - Provides conversation history and context
   - Enables personalized responses based on past interactions
   - Improves customer experience through continuity

6. **OpenTelemetry Observability**
   - Track and monitor Strands Agent performance and interactions
   - Provides insights into agent behavior and response quality
   - Enables observability for continuous improvement through monitoring

### Use Cases Demonstrated

- Customer profile retrieval
- Warranty status checking
- Solar system performance analysis
- Product Information and support queries
- Support ticket creation and information
- Previous conversations recall
- Guardrail activation

## Next Steps

To extend this example you can augment the current agent's capabilities with web search to include real time information about solar panel rules and regulations into the Strands Agent.



